<a href="https://colab.research.google.com/github/Nunitz/Nyvia/blob/main/Proyecto_de_ROP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from prophet import Prophet

In [ ]:
df = pd.read_csv("entrenamiento.csv")

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

Para nuestro proyecto solo requeriremos los dos últimos años y elegiremos la tienda con mayores ventas



In [ ]:
par2 = pd.DataFrame(df[(df['date'].dt.year>=2016)&(df['store']==2)])

Elegiremos el top 10 items de la tienda 2


In [ ]:
par3 = par2[par2["item"].isin([15, 45, 28, 8, 36, 18, 12, 50, 2, 24])]

Se hará la predicción por cada SKU

In [ ]:
models = {}

for sku in par3['item'].unique():

    sku_df = par3[par3['item'] == sku]
    sku_df = sku_df[['date', 'sales']]
    sku_df.columns = ['ds', 'y']

    model = Prophet()
    model.fit(sku_df)

    models[sku] = model

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Se unirán los modelos en una sola tabla para crear correctamente el reorder point

In [ ]:
all_forecasts = []

for sku, model in models.items():
    future = model.make_future_dataframe(periods=365)
    forecast = model.predict(future)
    forecast['sku'] = sku
    all_forecasts.append(forecast)

final_forecast = pd.concat(all_forecasts)

Redondearemos el yhat para que sea realista   


In [246]:
final_forecast["yhat"] = final_forecast["yhat"].round().astype(int)

Escogeremos el último año para generar el Rop

In [247]:
future_only = final_forecast[final_forecast["ds"] > par3["date"].max()]

Nos quedaremos con la fecha, el sku y las ventas


In [248]:
future_only = future_only[["ds", "sku", "yhat"]]

Sacaremos la demanda diaria

In [234]:
hist_avg = future_only.groupby(['sku'])['yhat'].mean().reset_index()
hist_avg.columns = ['sku', 'hist_avg_daily']

Igualmente sacaremos la desviación estandar del grupo


In [235]:
hist_std = future_only.groupby("sku")["yhat"].std().reset_index()
hist_std.columns = ['sku', 'hist_std_daily']

Uniremos las dos tablas para crear una parte del ROP

In [237]:
inventory_stats = hist_avg.merge(hist_std, on='sku')

Nuestro lead time será de 7 días y nuesta Z de 1.65

In [239]:
inventory_stats["lead_time"] = 7
inventory_stats["Z"] = 1.65

Y por último sacaremos el ROP

In [240]:
import numpy as np

inventory_stats["reorder_point"] = (
    inventory_stats["hist_avg_daily"] * inventory_stats["lead_time"]
    + inventory_stats["Z"] * inventory_stats["hist_std_daily"] * np.sqrt(inventory_stats["lead_time"])
)

In [241]:
inventory_stats

,sku,hist_avg_daily,hist_std_daily,lead_time,Z,reorder_point
0,2,86.616438,18.179531,7,1.65,685.677621
1,8,116.109589,24.185388,7,1.65,918.348186
2,12,104.304110,22.041372,7,1.65,826.350151
3,15,131.063014,27.425000,7,1.65,1037.164651
4,18,128.769863,26.964092,7,1.65,1019.100508
5,24,101.394521,20.284189,7,1.65,798.312061
6,28,133.668493,27.697289,7,1.65,1056.591680
7,36,116.320548,23.393404,7,1.65,916.367500
8,45,120.931507,25.118799,7,1.65,956.176407
9,50,97.813699,20.804647,7,1.65,775.518361


In [244]:
# Pedir datos al usuario
sku_input = int(input("Ingresa el SKU: "))
inventario_input = int(input("Ingresa el inventario actual: "))

# Buscar el ROP del SKU
sku_data = inventory_stats[inventory_stats["sku"] == sku_input]

if sku_data.empty:
    print("❌ SKU no encontrado")
else:
    rop = sku_data["reorder_point"].values[0]

    print(f"ROP para SKU {sku_input}: {rop}")

    if inventario_input <= rop:
        print("⚠️ ALERTA: Debes comprar inventario")
    else:
        print("✅ Inventario suficiente")

Ingresa el SKU: 2
Ingresa el inventario actual: 500
ROP para SKU 2: 685.6776214820292
⚠️ ALERTA: Debes comprar inventario
